# Introducción

Odoo es un sistema de código abierto. Esto nos permite modificar o implementar nuevas funciones. Para que esas personalizaciones no desaparezcan cuando Odoo se actualiza a una nueva versión, se implementan a través de módulos independientes.

A través de la programación de esos módulos es como normalmente se personaliza Odoo, pero sus características y datos también están disponibles a través de una API externa. Esta API, desarrollada en varios lenguajes, como Python, Ruby, PHP o Java, nos permiten interactuar con el sistema desde cualquier otra aplicación o página web que desarrollemos.

La API externa está disponible a través del uso del protocolo llamado XML-RPC. Este protocolo, bastante utilizado para el desarrollo de APIs, realiza peticiones remotas sobre un sistema (remote procedure call o RPC) usando XML para codificar esas peticiones y HTTP como mecanismo de transporte de las mismas.

# Conexión con la API

Para poder realizar la conexión, necesitamos los siguientes parámetros de nuestra instancia de Odoo:


* **URL**: Por ejemplo, `https://edu-test.odoo.com`.
*   **Nombre de la BD**: Por ejemplo, `edu-test`.
* **Nombre de usuario**: El correo electrónico con el que iniciamos sesión en nuestra instancia. Por ejemplo, `abcdefg123@g.educaand.es`.
* **Contraseña del usuario**: Por ejemplo, `password`.

**Importante**: Cuando damos de alta una instancia dominio.odoo.com, la contraseña que tenemos es de la cuenta en odoo.com, pero no se genera automáticamente una contraseña para el usuario administrador. Por lo tanto, tenemos que generar una (aconsejo que sea la misma, para evitar confusiones posteriores). Para ello, seguimos los siguientes pasos:


1.   Iniciamos sesión en nuestra instancia de Odoo.
2.   Pulsamos en la aplicación `Ajustes`.
3. En el apartado `OPCIONES GENERALES` → `Usuarios`, pulsamos en `Administrar usuarios`.
4. Hacemos clic en nuestro usuario.
5. Pulsamos en `Acción` → `Cambiar la contraseña`.
6. Establecemos la contraseña que queramos.

Una vez lo hemos hecho, ya podemos almacenar en variables los datos necesarios para la conexión.

Modifica el siguiente bloque de código con los datos de tu instancia y ejecútalo.




In [1]:
url = 'https://edu-test.odoo.com'
db = 'edu-test'
username = 'abcdefg123@g.educaand.es'
password = 'password'

A continuación, comprobaremos que nuestro servidor está activo. Importamos la librería `xmlrpc.client`, que nos permitirá conectarnos en Python usando el protocolo XML-RPC. El endpoint `xmlrpc/2/common` nos permite realizar peticiones que no requieren autenticación, como obtener información sobre la versión del sistema. Ejecuta el siguiente bloque de código para hacerlo.

In [2]:
import xmlrpc.client
common = xmlrpc.client.ServerProxy('{}/xmlrpc/2/common'.format(url))
common.version()

{'protocol_version': 1,
 'server_serie': '15.0',
 'server_version': '15.0+e',
 'server_version_info': [15, 0, 0, 'final', 0, 'e']}

Una vez comprobado que el servidor está activo, usaremos el mismo endpoint para identificarnos en el mismo usando los datos almacenados en las variables que hemos definido anteriormente. Esta petición devuelve un identificador de usuario (`uid`) que utilizaremos en las peticiones posteriores para demostrar que nos hemos autenticado.

In [3]:
uid = common.authenticate(db, username, password, {})

# Realizando peticiones a la API

El siguiente endpoint que vamos a utilizar es `xmlrpc/2/object`, utilizado para hacer peticiones a los métodos de los modelos de Odoo usando la función de RPC `execute_kw`.

Cada llamada a execute_kw necesita los siguientes parámetros:

*   La base de datos sobre la que se realiza la petición (un string).
*   El identificador de usuario (obtenido en el bloque de código anterior, un entero).
* La contraseña de ese usuario (un string).
* El nombre del modelo (un string).
* El nombre del método (un string).
* Un array o lista de los parámetros pasados por posición.
* Un diccionario de parámetros pasados usando palabras claves o keywords (opcional).

Por ejemplo, podemos leer el modelo `res.partner` llamando al método `check_access_rights` con la operación de lectura y el parámetro `raise_exception` para que nos devuelva `False` si ocurre un error con la petición y, por lo tanto, `True` si la petición se realiza correctamente.





In [4]:
models = xmlrpc.client.ServerProxy('{}/xmlrpc/2/object'.format(url))
models.execute_kw(db, uid, password,
    'res.partner', 'check_access_rights',
    ['read'], {'raise_exception': False})

True

## Listar los registros

Los registros pueden listarse y filtrarse usando `search()`.

`search()` utiliza un filtro y devuelve los identificadores en la base de datos de todos los registros que coincidan con el filtro. Por ejemplo, para listar las compañías clientes:

In [6]:
models.execute_kw(db, uid, password,
    'res.partner', 'search',
    [[['is_company', '=', True]]])

[14, 10, 11, 15, 41, 1, 12, 13, 9]

## Paginación

Por defecto, search devuelve todos los registros que coincidan con la condición, que pueden ser muchos. Podemos usar `offset` (comenzar a listar a partir de un número de registros) y `limit` (limitar el número de registros) para obtener un subconjunto de los mismos únicamente.

In [7]:
models.execute_kw(db, uid, password,
    'res.partner', 'search',
    [[['is_company', '=', True]]],
    {'offset': 6, 'limit': 2})

[12, 13]

## Contar registros

En lugar de obtener una lista gigante de identificadores de registros, se puede utilizar `search_count()` para obtener únicamente el número de ellos que coinciden con el filtro.

In [ ]:
models.execute_kw(db, uid, password,
    'res.partner', 'search_count',
    [[['is_company', '=', True]]])

9

## Leer registros

Los datos de un registro están accesibles a través del método `read()`, que coge una lista de identificadores (`ids`) devueltos por `search()` y, opcionalmente, una lista de campos que obtener. Por defecto, obtendrá todos los campos que el usuario tenga permisos para leer, que suelen ser bastantes.

In [ ]:
ids = models.execute_kw(db, uid, password,
    'res.partner', 'search',
    [[['is_company', '=', True]]],
    {'limit': 1})
[record] = models.execute_kw(db, uid, password,
    'res.partner', 'read', [ids])
# cuenta el número de campos obtenidos por defecto
len(record)

151

Es mejor obtener solamente los campos que nos interesen.

In [ ]:
models.execute_kw(db, uid, password,
    'res.partner', 'read',
    [ids], {'fields': ['name', 'country_id', 'comment']})

[{'comment': False,
  'country_id': [233, 'United States'],
  'id': 14,
  'name': 'Azure Interior'}]

## Listar los campos de registros

El método `fields_get()` puede utilizarse para inspeccionar los campos de un modelo y comprobar cuáles nos interesan.

Como devuelve una gran cantidad de meta-información del modelo, debería filtrarse antes de mostrarse por pantalla. Los más interesantes para la lectura por parte de un ser humano son `string` (la etiqueta del campo), `help` (un texto de ayuda sobre el mismo si está disponible) y `type` (para conocer qué tipo de valores esperar o saber qué valores enviar para actualizar un registro).

In [ ]:
models.execute_kw(
    db, uid, password, 'res.partner', 'fields_get',
    [], {'attributes': ['string', 'help', 'type']})

{'__last_update': {'string': 'Last Modified on', 'type': 'datetime'},
 'active': {'string': 'Active', 'type': 'boolean'},
 'active_lang_count': {'string': 'Active Lang Count', 'type': 'integer'},
 'activity_calendar_event_id': {'string': 'Next Activity Calendar Event',
  'type': 'many2one'},
 'activity_date_deadline': {'string': 'Next Activity Deadline',
  'type': 'date'},
 'activity_exception_decoration': {'help': 'Type of the exception activity on record.',
  'string': 'Activity Exception Decoration',
  'type': 'selection'},
 'activity_exception_icon': {'help': 'Icon to indicate an exception activity.',
  'string': 'Icon',
  'type': 'char'},
 'activity_ids': {'string': 'Activities', 'type': 'one2many'},
 'activity_state': {'help': 'Status based on activities\nOverdue: Due date is already passed\nToday: Activity date is today\nPlanned: Future activities.',
  'string': 'Activity State',
  'type': 'selection'},
 'activity_summary': {'string': 'Next Activity Summary', 'type': 'char'},
 '

## Búsqueda y lectura

Como suele ser una tarea habitual, Odoo proporciona el método `search_read()` que es equivalente a hacer un método `search()` y luego un método `read()` para esa búsqueda.

Los argumentos son similares a `search()`, pero también se le pasa una lista de campos (como en `read()`, si no se le pasa, mostrará todos los campos).

In [ ]:
models.execute_kw(db, uid, password,
    'res.partner', 'search_read',
    [[['is_company', '=', True]]],
    {'fields': ['name', 'country_id', 'comment'], 'limit': 5})

[{'comment': False,
  'country_id': [233, 'United States'],
  'id': 14,
  'name': 'Azure Interior'},
 {'comment': False,
  'country_id': [233, 'United States'],
  'id': 10,
  'name': 'Deco Addict'},
 {'comment': False,
  'country_id': [233, 'United States'],
  'id': 11,
  'name': 'Gemini Furniture'},
 {'comment': False,
  'country_id': [233, 'United States'],
  'id': 15,
  'name': 'Lumber Inc'},
 {'comment': False,
  'country_id': [233, 'United States'],
  'id': 41,
  'name': 'My Company (Chicago)'}]

## Crear registros

Los registros de un modelo se crean usando `create()`. El método creará un solo registro y devolverá su identificador en la base de datos.

A `create()` se le pasa un diccionario de campos y valores, usados para inicializar el registro. Los campos a los que no se le indique un valor, se les pondrá el valor por defecto (si lo tuviesen).

In [11]:
id = models.execute_kw(db, uid, password, 'res.partner', 'create', [{
    'name': "New Partner",
}])
print(id)

48


## Actualizar registros

Los registros pueden actualizarse usando `write()`, al que se le pasa una lista de los registros a actualizar y un diccionario de campos con los valores similar a como se hace en `create()`.

Se pueden actualizar múltiples registros al mismo tiempo, pero obtendrán los mismos valores para los campos que se estén actualizando. No es posible de momento realizar actualizaciones en función del valor de otro campo de un registro.

In [12]:
models.execute_kw(db, uid, password, 'res.partner', 'write', [[id], {
    'name': "Newer partner"
}])
# obtener el nombre del registro después de haberlo actualizado
models.execute_kw(db, uid, password, 'res.partner', 'name_get', [[id]])

[[48, 'Newer partner']]

## Eliminar de registros

Los registros pueden eliminarse de forma masiva pasando sus identificadores a `unlink()`.

In [13]:
models.execute_kw(db, uid, password, 'res.partner', 'unlink', [[id]])
# comprobamos si el registro eliminado sigue en la base de datos
models.execute_kw(db, uid, password,
    'res.partner', 'search', [[['id', '=', id]]])

[]